In [36]:
import numpy as np,matplotlib.pyplot as plt,mne,os,sys,glob,numpy as np,torch
from mne.datasets import eegbci
from IPython.display import clear_output
from torch import nn
import torch.optim as optim
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal
import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from mne import (io, compute_raw_covariance, read_events, pick_types, Epochs)
from mne.preprocessing import Xdawn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import pandas as pd
import seaborn as sns
import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test

In [37]:
#X_syntheic = np.load("Synthetic_data_V2/X_subject_7_72_['left', 'right', 'foot', 'tongue'].npy")
#y_syntheic = np.load("Synthetic_data_V2/y_subject_7_72_['left', 'right', 'foot', 'tongue'].npy")


X_syntheic = np.load("Synthetic_data_more/X_subject_1_72_['left', 'right', 'foot', 'tongue'].npy")
y_syntheic = np.load("Synthetic_data_more/y_subject_1_72_['left', 'right', 'foot', 'tongue'].npy")

FileNotFoundError: [Errno 2] No such file or directory: "Synthetic_data_more/X_subject_1_72_['left', 'right', 'foot', 'tongue'].npy"

In [ ]:
from mne.io import RawArray
freqs = np.arange(8, 14)  # frequencies from 2-35Hz
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
baseline = (0, 0.025)  # baseline interval (in s)
tmin, tmax = 0, 4
freq_bounds = {'_': 0,
               'delta': 3,
               'theta': 7,
               'alpha': 13,
               'beta': 35,
               'gamma': 140}
freq_bands_of_interest = ['delta', 'theta', 'alpha', 'beta']


ch_names = ["Fz",
 'FC3','FC1','FCZ','FC2','FC4',
 'C5','C3','C1','Cz','C2','C4','C6',
 'CP3','CP1','CPz','CP2','CP4',
 'P1','Pz','P2',
 'POZ'
 ]

In [ ]:
info = mne.create_info(ch_names=ch_names,sfreq = 250 ,ch_types ="eeg")
epochs = mne.EpochsArray(X_syntheic,info,baseline=baseline)
for i in range(y_syntheic.shape[0]):
    epochs.events[i][2] = y_syntheic[i]
epochs.event_id = {'0':0,'1':1,'2':2,'3':3}

epochs.pick_channels(["C3","C4"])
cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
kwargs = dict(n_permutations=100, step_down_p=0.05, seed=1,
                  buffer_size=None, out_type='mask')
tfr = tfr_multitaper(epochs, freqs=freqs, n_cycles=freqs, use_fft=True,return_itc=False, average=False, decim=1)
tfr.crop(tmin, tmax).apply_baseline(baseline, mode="percent")
df = tfr.to_data_frame(time_format=None, long_format=True)
df['band'] = pd.cut(df['freq'], list(freq_bounds.values()),
                    labels=list(freq_bounds)[1:])



df = df[df.band.isin(freq_bands_of_interest)]

df['band'] = df['band'].cat.remove_unused_categories()

# Order channels for plotting:
df['channel'] = df['channel'].cat.reorder_categories(('C3', 'C4'),
                                                     ordered=True)

g = sns.FacetGrid(df, row='band', col='condition')
g.map(sns.lineplot, 'time', 'value', 'channel', n_boot=10)
axline_kw = dict(color='black', linestyle='dashed', linewidth=0.5, alpha=0.5)
g.map(plt.axhline, y=0, **axline_kw)
g.map(plt.axvline, x=0, **axline_kw)
g.set(ylim=(-1, None))
g.set_axis_labels("Time (s)", "ERDS (%)")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.add_legend(ncol=2, loc='lower center')
g.fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.08)

#g.savefig("ERD/{0}_ERD_S{1}.png".format(runs_name[i],subjects[0]))
#g.savefig("unicorn/{0}_ERD_S{1}.png".format(runs_name[i],subjects[0]))
g.fig#.clf()